In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\Documents\\Brain-Tumor-Classification-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Dell\\Documents\\Brain-Tumor-Classification-Project'

In [12]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size : list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [13]:
from braintumorClassifier.constants import *
from braintumorClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [14]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAM_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'data')
        create_directories([
            Path(training.root_dir)
        ])
        print("Before TrainingConfig initialization")


        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE,
        )
        return training_config
        print("After TrainingConfig initialization")

    

In [15]:
import time 
import os
import urllib.request as request
from zipfile import ZipFile
# import tensorflow as tf

In [16]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}',
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [25]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            class_mode = 'categorical'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40, 
                horizontal_flip = True, 
                width_shift_range = 0.2, 
                height_shift_range = 0.2, 
                shear_range = 0.2, 
                zoom_range = 0.2, 
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        self.train_generator = train_datagenerator.flow_from_directory(directory = self.config.training_data,
                                                                            subset = 'training', shuffle = True,
                                                                            **dataflow_kwargs )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.step_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator, 
            epochs = self.config.params_epochs, 
            steps_per_epoch = self.step_per_epoch,
            validation_data = self.valid_generator,
            validation_steps = self.validation_steps,
            callbacks = callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [26]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list)
except Exception as e:
    raise e


[2024-02-10 01:05:48,847: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-10 01:05:48,849: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-10 01:05:48,850: INFO: common: created directory at: artifacts]
[2024-02-10 01:05:48,852: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-02-10 01:05:48,852: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-02-10 01:05:48,855: INFO: common: created directory at: artifacts\training]
Before TrainingConfig initialization
Found 208 images belonging to 2 classes.
Found 833 images belonging to 2 classes.
Epoch 1/10
[2024-02-10 01:05:49,829: WARNING: module_wrapper: From c:\Users\Dell\anaconda1\envs\Project1\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
52/52 [==============================] - 174s 3s/step - loss: 15.7

c:\Users\Dell\anaconda1\envs\Project1\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


52/52 [==============================] - 182s 4s/step - loss: 13.2330 - accuracy: 0.5471 - val_loss: 22.1806 - val_accuracy: 0.5000
Epoch 3/10
52/52 [==============================] - 176s 3s/step - loss: 12.1738 - accuracy: 0.5606 - val_loss: 5.4698 - val_accuracy: 0.6538
Epoch 4/10
52/52 [==============================] - 178s 3s/step - loss: 12.3788 - accuracy: 0.5667 - val_loss: 5.8129 - val_accuracy: 0.6538
Epoch 5/10
52/52 [==============================] - 174s 3s/step - loss: 9.9271 - accuracy: 0.5973 - val_loss: 9.6736 - val_accuracy: 0.5192
Epoch 6/10
52/52 [==============================] - 175s 3s/step - loss: 9.1457 - accuracy: 0.6169 - val_loss: 17.1290 - val_accuracy: 0.5048
Epoch 7/10
52/52 [==============================] - 180s 3s/step - loss: 8.4105 - accuracy: 0.6218 - val_loss: 12.2169 - val_accuracy: 0.5240
Epoch 8/10
52/52 [==============================] - 176s 3s/step - loss: 9.9495 - accuracy: 0.5863 - val_loss: 3.8725 - val_accuracy: 0.7692
Epoch 9/10
52/52 [